In [1]:
# Project Libraries
import data

# Generic Libraries
import numpy as np
import pandas as pd

Data Frames:
- Orderbook with previous operations
- Liquidity pools history (deliverable)
- Trade History (deliverable) 

In [134]:
# ----- Useful dataframes

def dataframes():
    """
    L

    Parameters
    ----------

    L

    Returns
    -------

    L
    
    """
    
    # Informative dataframe
    df_info = pd.DataFrame({
        'bid_volume': [0.0001],
        'ask_volume': [0.0001],
        'commission': [0.000025]
    })

    # Oders dataframe
    df_orders = pd.DataFrame(columns = ['timestamp', 'order_id', 'side', 'price', 'order_amount', 'symbol'])

    # Trades dataframe
    df_trades = pd.DataFrame(columns = ['timestamp', 'order_id', 'trade_id', 'side', 'price', 'symbol', 'filled_amount', 'order_amount', 'fee'])

    # Inventory dataframe
    df_inventory = pd.DataFrame({
        'timestamp': [0],
        'base (BTC)': [5],
        'quote (USDT)': [500000]
    })

    # Rebalance dataframe
    df_rebalances = pd.DataFrame(columns = ['timestamp', 'sent', 'received', 'price', 'fee'])

    return df_info, df_orders, df_trades, df_inventory, df_rebalances


In [126]:
def order_id(col):
    if col == 'order_id':
        try:
            return df_orders['order_id'].iloc[-1] + 1
        except:
            return 1
    if col == 'index':
        try:
            return df_orders.index[-1] + 1
        except:
            return 0

In [127]:
# ----- Order Placement Function
def order_placement(df, ts, position, order_id_passed):
    i = order_id('index')
    if position == 'buy':
        df_orders.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
        df_orders.at[i, 'order_id'] = order_id_passed
        df_orders.at[i, 'side'] = 'buy' 
        df_orders.at[i, 'price'] = df['bid'].iloc[ts]
        df_orders.at[i, 'order_amount'] = 0.0001
        df_orders.at[i, 'symbol'] = 'BTC/USDT'
    else:
        df_orders.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
        df_orders.at[i, 'order_id'] = order_id_passed
        df_orders.at[i, 'side'] = 'sell'
        df_orders.at[i, 'price'] = df['bid'].iloc[ts]       
        df_orders.at[i, 'order_amount'] = 0.0001
        df_orders.at[i, 'symbol'] = 'BTC/USDT'

In [129]:
# Position valuation
def position_valuation(size_variation: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation > 0:
            current_position = 0
        elif price_variation == 0:
            if size_variation > 0:
                current_position = constant_position_base
            else:
                if size_variation <= constant_position_base:
                    current_position = constant_position_base - size_variation
                else:
                    current_position = 0
        else:
            current_position = constant_position_base
    # Bid
    if operation == 'bid':
        if price_variation < 0:
            current_position = 0
        elif price_variation == 0:
            if size_variation > 0:
                current_position = constant_position_base
            else:
                if size_variation <= constant_position_base:
                    current_position = constant_position_base - size_variation
                else:
                    current_position = 0
        else:
            current_position = constant_position_base
    return current_position

In [141]:
def trade_id(col):
    if col == 'trade_id':
        try:
            return df_trades['trade_id'].iloc[-1] + 1
        except:
            return 1
    if col == 'index':
        try:
            return df_trades.index[-1] + 1
        except:
            return 0

In [204]:
def trade(df, ts, position, trade_id_passed, filled_volume):

    i = trade_id('index')
    position_orders = ('buy' if position == 'bid' else 'sell')
    traded_price = df_orders[df_orders['side'] == position_orders].iloc[-1]['price']
    ordered_amount = df_orders[df_orders['side'] == position_orders].iloc[-1]['order_amount']
    commision = df_info['commission'].iloc[0]

    df_trades.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
    df_trades.at[i, 'order_id'] = df_orders[df_orders['side'] == position_orders].iloc[-1]['order_id']
    df_trades.at[i, 'trade_id'] = trade_id_passed
    df_trades.at[i, 'side'] = ('buy' if position == 'bid' else 'sell')
    df_trades.at[i, 'price'] = traded_price
    df_trades.at[i, 'symbol'] = 'BTC/USDT'
    df_trades.at[i, 'filled_amount'] = filled_volume
    df_trades.at[i, 'order_amount'] = ordered_amount
    df_trades.at[i, 'fee'] = traded_price * ordered_amount * commision


In [173]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})


In [209]:
#df_info, df_orders, df_trades, df_inventory, df_rebalances = fn.dataframes()
df_info, df_orders, df_trades, df_inventory, df_rebalances = dataframes()

order_placement(df = df_bitfinex_tob,
                ts = 0,  
                position = "buy",
                order_id_passed = order_id("order_id"))
order_placement(df = df_bitfinex_tob, 
                ts = 0,
                position = "sell",
                order_id_passed = order_id("order_id"))

volume_fill_ask =  0

iteration = 1

for op in ['bid','ask']:
    buy_sell = ('buy' if op == 'bid' else 'sell')
    filled_amount = position_valuation(
        size_variation  = df_bitfinex_tob[op+'_size'].iloc[iteration] - df_bitfinex_tob[op+'_size'].iloc[iteration - 1],
        price_variation = df_bitfinex_tob[op].iloc[iteration] - df_orders[df_orders['side'] == buy_sell].iloc[0]['price'],
        operation = op,
        constant_position_base = df_orders[df_orders['side'] == buy_sell].iloc[0]['order_amount'] - volume_fill_ask
        )

    trade( df = df_bitfinex_tob,
        ts = iteration,
        position = buy_sell, 
        trade_id_passed = trade_id('trade_id'), 
        filled_volume = filled_amount
        )
    position_trades = df_trades.copy()
    position_trades = position_trades[position_trades['side'] == buy_sell]
    if position_trades['filled_amount'].iloc[-1] == position_trades['order_amount'].iloc[-1]:
        order_placement(df = df_bitfinex_tob, 
                        ts = iteration, 
                        position = op,
                        order_id_passed = order_id("order_id"))

df_trades


IndexError: single positional indexer is out-of-bounds

In [178]:
df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,1,buy,28282.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,2,sell,28282.0,0.0001,BTC/USDT
2,2021-07-05 13:07:00+00:00,3,sell,28323.0,0.0001,BTC/USDT


In [61]:
# Explanation Missing
def position_valuation(size: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation >= 0:
            if ( constant_position_base - size ) < 0:
                current_position = 0
            else:
                current_position = constant_position_base - size
        else:
            current_position = constant_position_base
    # Bid
    if operation == 'bid':
        if price_variation <= 0:
            if ( constant_position_base - size ) < 0:
                current_position = 0
            else:
                current_position = constant_position_base - size
        else:
            current_position = constant_position_base
    return current_position